<div class="alert alert-block alert-success">

<b> Exercise 1: <br> Calibrate the Cepheid P-L relation in the Milky Way with Gaia </b>

</div>

Here we import packages:

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
import astropy.units as u
from uncertainties import unumpy as unp
from uncertainties import ufloat

Type here the path where you placed the notebook and data files:

In [ ]:
path_data = '/Users/louise/Desktop/2025-04-24_CosmoVerse_Lecture_Cepheids/Tutorials/'

First, we read the data file with Cepheid information from Riess et al. (2021): <br> (https://ui.adsabs.harvard.edu/abs/2021ApJ...908L...6R/abstract) <br> <br>
Columns are: <br> - Cepheid name, <br> - log(period), <br> - apparent magnitudes in HST filters F555W, F814W, F160W and respective errors, <br> - "mHW" Wesenheit magnitdue (see definition below) and error, <br> - metallicity (Fe/H), <br> - "plx_exp" expected parallax and error, <br> - "plx_EDR3" Gaia DR3 parallax and error.  <br> <br> 

In [ ]:
R21 = pd.read_csv(path_data + 'Riess2021_tab1.dat', sep='\s+', header=0) 
print(R21)

<b> Note: </b> <br> In this exercise we calibrate the Cepheid Period-Luminosity relation in the HST near infrared filter F160W, which is similar to the H-band filter. <br> 
Although this filter is very little affected by dust (compared to optical filters), the F160W magnitude needs to be corrected for interstellar extinction. <br>
Therefore, we adopt a "corrected" F160W magnitude called the "Wesenheit" index (mHW), defined as: <br> 

<b> mHW = F160W - 0.386 * (F555W - F814W) </b> <br> 

where we subtract a color term to the F160W apparent magnitude. <br> 
In the following we define a list of logP, apparent magnitudes "app_mag" with their errors, and Gaia DR3 parallaxes with their errors for this Cepheid sample.

In [ ]:
logP = list(R21['logP'])
app_mag,  e_app_mag  = list(R21['mHW']),     list(R21['e_mHW'])
parallax, e_parallax = list(R21['plx_DR3']), list(R21['e_plx_DR3'])

Knowing the distance (d ~ 1/plx) of each Cepheid, we now derive their absolute magnitudes "M": <br> <br>

<b> M = mHW + 5*log(plx) - 10  </b> <br>

where "mHW" is the apparent magnitude and "plx" is the parallax in milli-arcsec (as given in the data table). <br>
We propagate the errors from apparent magnitudes and from paralalxes into the absolute magnitude error.

In [ ]:
abs_mag_with_uncert = [ufloat(app_mag[i], e_app_mag[i]) + 5*unp.log10(ufloat(parallax[i], e_parallax[i])) -10 for i in range(len(R21))]

abs_mag   = [M.nominal_value for M in abs_mag_with_uncert]
e_abs_mag = [M.std_dev       for M in abs_mag_with_uncert]

Now that we know the logP and absolute luminosity of each Cepheid, we can plot them and see if they verify a Period-Luminosity relation:

In [ ]:
plt.figure(figsize=(6,4))
plt.subplots_adjust(left=0.10, right=0.96, top=0.92, bottom=0.12, hspace=0.1, wspace=0.3)

plt.errorbar(logP, abs_mag, yerr=e_abs_mag, fmt='o', color='blue', markerfacecolor='k', markeredgewidth=0.8, markersize=6, capsize=0, ecolor='k', elinewidth=0.7)

plt.xlim(0.3, 1.9)
plt.ylim(-8.7, -3.4)
plt.gca().invert_yaxis()
plt.xlabel('logP (days)', fontsize=12)
plt.ylabel('Absolute Magnitude (mag)', fontsize=12)
plt.show()

We can now fit the Period-Luminosity relation with a simple linear form: <b> M = a*logP + b</b>

In [ ]:
popt, pcov = curve_fit(lambda X, A, B: A*X+B, logP, abs_mag, sigma=e_abs_mag)
slope, intercept = popt[0], popt[1]
e_slope, e_intercept = np.sqrt(np.diag(pcov))[0], np.sqrt(np.diag(pcov))[1]

print(f' -> M = ({slope:.2f} ± {e_slope:.2f})*logP  - ({-intercept:.2f} ± {e_intercept:.2f}) ')

<b> This relation is the absolute calibration of the Period-Luminosity relation in the Milky Way. </b>

We plot the PL relation again, now with the fit:

In [ ]:
xfine = np.linspace(0.2, 2.4, 100) 

plt.figure(figsize=(6,4))
plt.subplots_adjust(left=0.10, right=0.96, top=0.92, bottom=0.12, hspace=0.1, wspace=0.3)

plt.plot(xfine, slope*xfine + intercept, '-b', linewidth=1)
plt.errorbar(logP, abs_mag, yerr=e_abs_mag, fmt='o', color='blue', markerfacecolor='k', markeredgewidth=0.8, markersize=6, capsize=0, ecolor='k', elinewidth=0.7)

plt.xlim(0.3, 1.9)
plt.ylim(-8.7, -3.4)
plt.gca().invert_yaxis()
plt.xlabel('logP (days)', fontsize=12)
plt.ylabel('Absolute Magnitude (mag)', fontsize=12)
plt.show()